In [ ]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchsummary
import numpy as np
from PIL import Image
from torchvision import datasets, transforms
import random
import time
import zipfile
import subprocess
import json

In [ ]:
def seed_everything(seed=42):
    """
    모든 랜덤 시드를 주어진 값으로 고정합니다.
    """
    random.seed(seed)  # Python random 모듈
    np.random.seed(seed)  # Numpy 랜덤 시드
    torch.manual_seed(seed)  # PyTorch 랜덤 시드
    torch.cuda.manual_seed(seed)  # GPU를 위한 PyTorch 랜덤 시드
    torch.cuda.manual_seed_all(seed)  # 멀티 GPU를 위한 PyTorch 랜덤 시드

seed_everything()

In [ ]:
# class CustomDataset(torch.utils.data.Dataset):
#     def __init__(self, image_zip_paths, label_zip_paths, cache_dir, transform=None, workers=4):
#         super().__init__()
#         self.image_zip_paths = image_zip_paths
#         self.label_zip_paths = label_zip_paths
#         self.cache_dir = cache_dir
#         self.transform = transform
#         self.workers = workers

#         if not os.path.exists(self.cache_dir):
#             os.makedirs(self.cache_dir, exist_ok=True)

#         self.image_names = {}
#         self.label_names = {}
#         self.image_list = []

#         self._prepare()

#     def _extract_and_cache(self, zip_path, file_name, cache_file_path, attempts=3):
#         if os.path.exists(cache_file_path):
#             return True

#         command = ["unzip", "-o", zip_path, file_name, "-d", os.path.dirname(cache_file_path)]

#         for attempt in range(attempts):
#             result = subprocess.run(command, check=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#             # 성공적으로 압축 해제되었거나, 경고만 있는 경우
#             if result.returncode == 0 or "warning" in result.stderr.decode().lower():
#                 return True
#             else:
#                 # 오류 메시지가 있고, 재시도 횟수가 남아있는 경우
#                 print(f"Attempt {attempt + 1}: Error unzipping file => {zip_path}")
#                 time.sleep(2 ** attempt)

#         # 모든 시도가 실패한 경우
#         print(f"Failed to extract {file_name}")
#         return False

#     def _prepare(self):
#         max_tries = 3
#         wait_seconds = 10

#         # 각 zip 파일에 대한 재시도 횟수를 추적
#         retries = {zip_path: 0 for zip_path in self.image_zip_paths + self.label_zip_paths}

#         # 전체 압축 파일 리스트
#         to_process = [(zip_path, '.png') for zip_path in self.image_zip_paths] + [(zip_path, '.json') for zip_path in self.label_zip_paths]

#         while to_process:
#             # 병렬처리
#             with ThreadPoolExecutor(max_workers=self.workers) as executor:
#                 future_to_zip_path = {
#                     executor.submit(self._process_zip, zip_path, file_extension): (zip_path, file_extension)
#                     for zip_path, file_extension in to_process
#                 }

#                 # 처리 중 예외 발생 시 재시도할 작업 목록
#                 to_retry = []

#                 # future 객체의 작업완료 상황을 모니터링
#                 for future in as_completed(future_to_zip_path):
#                     zip_path, file_extension = future_to_zip_path[future]

#                     try:
#                         success = future.result()
#                         if not success:
#                             raise Exception(f"Failed to process {zip_path}")

#                     except OSError as e: # 구글드라이브와의 연결로 인한 예외처리
#                         if e.errno == 107 and retries[zip_path] < max_tries:
#                             print(f"OSError [Errno 107] => {zip_path}")
#                             to_retry.append((zip_path, file_extension))
#                             retries[zip_path] += 1
#                             time.sleep(wait_seconds)  # 각 재시도 사이에 대기
#                             wait_seconds *= 2  # 대기 시간 증가
#                         else:
#                             print(f"Unexpected Error : {zip_path}: {e}")

#                     except Exception as e:
#                         print(f"Exception processing {zip_path}: {e}")

#                 # 재시도할 작업이 있으면 to_process 업데이트
#                 to_process = to_retry if to_retry else []

#         self.image_list = sorted(self.image_names.keys())

#     def _process_zip(self, zip_path, file_extension):
#         success = True
#         with zipfile.ZipFile(zip_path, 'r') as zip_ref:

#             for file_name in sorted(zip_ref.namelist()):

#                 if file_name.endswith(file_extension):
#                     base_name = os.path.splitext(os.path.basename(file_name))[0].lstrip('_')  # 확장자 제외 파일 기본 이름

#                     # 데이터 접근시 사용할 경로
#                     cache_file_path = os.path.join(self.cache_dir, os.path.basename(zip_path).replace('.zip', ''), file_name.replace('/', '_').lstrip('_'))

#                     if file_extension == '.png':
#                         self.image_names[base_name.lower()] = cache_file_path
#                     else:
#                         self.label_names[base_name.lower()] = cache_file_path

#                     extract_success = self._extract_and_cache(zip_path, file_name, cache_file_path)
#                     success = success and extract_success

#         return success

#     def __len__(self):
#         return len(self.image_names)

#     def __getitem__(self, idx):
#         # 파일 이름을 키 리스트에서 추출
#         base_name = self.image_list[idx]

#         image_path = self.image_names.get(base_name)

#         try:
#             image = Image.open(image_path)
#         except Exception as e:
#             return None

#         label_path = self.label_names.get(base_name)
#         with open(label_path, 'r') as f:
#             label_data = json.load(f)

#         age_past = label_data['age_past']
#         gender = label_data['gender']
#         box = label_data['annotation'][0]['box']

#         image = image.crop((box['x'], box['y'], box['x'] + box['w'], box['y'] + box['h']))

#         if self.transform:
#             image = self.transform(image)

#         gender_label = 0 if gender == 'male' else 1

#         label = {'age_past': age_past, 'gender': gender_label}

#         return image, label


In [ ]:
# 랜덤하게 적용
random_transforms = transforms.RandomApply([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
], p=0.5)

# 항상 적용
always_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.6284, 0.4901, 0.4325], std=[0.1869, 0.1712, 0.1561]),
])

transform_train = transforms.Compose([
    random_transforms,
    always_transforms
])

transform_val = transforms.Compose([
    always_transforms
])

In [ ]:
# def get_zip_files(directory):
#     return [os.path.join(directory, zip) for zip in os.listdir(directory) if zip.endswith('.zip')] # 리스트를 정렬하면 속도 하락

# train_images = '/content/drive/Othercomputers/Home/data_age/data/Training/image'
# train_labels = '/content/drive/Othercomputers/Home/data_age/data/Training/label'
# train_cache = '/content/cache/train'

# val_images = '/content/drive/Othercomputers/Home/data_age/data/Validation/image'
# val_labels = '/content/drive/Othercomputers/Home/data_age/data/Validation/label'
# val_cache = '/content/cache/val'

# train_image_zips = get_zip_files(train_images)
# train_label_zips = get_zip_files(train_labels)
# val_image_zips = get_zip_files(val_images)
# val_label_zips = get_zip_files(val_labels)

In [ ]:
# %time train_dataset = CustomDataset(train_image_zips, train_label_zips, train_cache, transform_train)

In [ ]:
# %time val_dataset = CustomDataset(val_image_zips, val_label_zips, val_cache, transform_val)

In [ ]:

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data_root, transform=None):
        super().__init__()
        self.transform = transform

        self.items = []  # (이미지 경로, 라벨 경로) 튜플을 저장

        # 모든 라벨 파일 경로를 소문자로 변환하여 저장
        label_paths_lower = {os.path.join(data_root, path).lower(): path for path in glob.glob(os.path.join(data_root, '*_*/*.json'))}

        image_paths = glob.glob(os.path.join(data_root, '*_*/*.png'))  # 이미지 파일 경로 수집
        for image_path in image_paths:
            base_name = os.path.splitext(os.path.basename(image_path))[0].lower()  # 파일명에서 확장자 제거 및 소문자 변환
            label_dir = os.path.dirname(image_path).replace('TS_', 'TL_').replace('VS_', 'VL_').lower()
            label_path = os.path.join(label_dir, base_name + '.json')

            # 소문자 변환된 라벨 경로로 실제 파일 경로 확인
            actual_label_path = label_paths_lower.get(label_path)
            if actual_label_path:
                self.items.append((base_name, (image_path, actual_label_path)))

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        base_name, (image_path, label_path) = self.items[idx]

        try:
            image = Image.open(image_path).convert('RGB')
        except Exception as e:
            return None

        with open(label_path, 'r') as f:
            label_data = json.load(f)

        age_past = label_data['age_past']
        gender = label_data['gender']
        box = label_data['annotation'][0]['box']
        image = image.crop((box['x'], box['y'], box['x'] + box['w'], box['y'] + box['h']))

        if self.transform:
            image = self.transform(image)

        gender_label = 0 if gender == 'male' else 1
        label = {'age_past': age_past, 'gender': gender_label}

        return image, label


In [ ]:
train_data_root = '/content/cache/train'
val_data_root = '/content/cache/val'
train_dataset =  CustomDataset(train_data_root, transform_train)
val_dataset =  CustomDataset(val_data_root, transform_val)

In [ ]:
def custom_collate_fn(batch):
    batch = [item for item in batch if item is not None]
    return torch.utils.data.dataloader.default_collate(batch)

In [ ]:
batch_size = 96

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    collate_fn=custom_collate_fn,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    prefetch_factor=8,
    pin_memory=True
)

val_loader = torch.utils.data.DataLoader(
    dataset=val_dataset,
    collate_fn=custom_collate_fn,
    batch_size=batch_size,
    num_workers=2,
    prefetch_factor=8,
    pin_memory=True
)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
class Age_Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1_1 = nn.Conv2d(3, 128, kernel_size=3, padding=1)
        self.bn1_1 = nn.BatchNorm2d(128)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn2_1 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn3_1 = nn.BatchNorm2d(512)
        self.conv3_1x1 = nn.Conv2d(512, 256, kernel_size=1)  # 1x1 컨볼루션 적용
        self.bn3_1x1 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 평균 풀링 레이어를 추가하여 마지막 컨볼루션 레이어의 출력 크기를 감소
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc1 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.5)  # 드롭아웃 비율 조정
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.bn1_1(self.conv1_1(x)))
        x = self.pool1(x)

        x = F.relu(self.bn2_1(self.conv2_1(x)))
        x = self.pool2(x)

        x = F.relu(self.bn3_1(self.conv3_1(x)))
        x = F.relu(self.bn3_1x1(self.conv3_1x1(x)))
        x = self.pool3(x)

        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(self.dropout(x)))
        x = self.fc2(x)
        x = x.squeeze()
        return x


In [ ]:
# # 배치정규화 적용
# class Age_Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1_1 = nn.Conv2d(3, 128, kernel_size=3)
#         self.bn1_1 = nn.BatchNorm2d(128)
#         self.conv1_2 = nn.Conv2d(128, 128, kernel_size=3)
#         self.bn1_2 = nn.BatchNorm2d(128)
#         self.conv1_3 = nn.Conv2d(128, 128, kernel_size=3)
#         self.bn1_3 = nn.BatchNorm2d(128)

#         self.pool = nn.MaxPool2d(kernel_size=3, stride=2)

#         self.conv2_1 = nn.Conv2d(128, 128, kernel_size=3)
#         self.bn2_1 = nn.BatchNorm2d(128)
#         self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3)
#         self.bn2_2 = nn.BatchNorm2d(128)
#         self.conv2_3 = nn.Conv2d(128, 128, kernel_size=3)
#         self.bn2_3 = nn.BatchNorm2d(128)

#         self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3)
#         self.bn3_1 = nn.BatchNorm2d(256)
#         self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3)
#         self.bn3_2 = nn.BatchNorm2d(256)
#         self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3)
#         self.bn3_3 = nn.BatchNorm2d(256)

#         self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3)
#         self.bn4_1 = nn.BatchNorm2d(512)
#         self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3)
#         self.bn4_2 = nn.BatchNorm2d(512)
#         self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3)
#         self.bn4_3 = nn.BatchNorm2d(512)

#         self.fc1 = nn.Linear(512 * 7 * 7, 512)
#         self.dropout = nn.Dropout(0.25)
#         self.fc2 = nn.Linear(512, 1)

#     def forward(self, x):
#         x = F.relu(self.bn1_1(self.conv1_1(x)))
#         x = F.relu(self.bn1_2(self.conv1_2(x)))
#         x = F.relu(self.bn1_3(self.conv1_3(x)))
#         x = self.pool(x)

#         x = F.relu(self.bn2_1(self.conv2_1(x)))
#         x = F.relu(self.bn2_2(self.conv2_2(x)))
#         x = F.relu(self.bn2_3(self.conv2_3(x)))
#         x = self.pool(x)

#         x = F.relu(self.bn3_1(self.conv3_1(x)))
#         x = F.relu(self.bn3_2(self.conv3_2(x)))
#         x = F.relu(self.bn3_3(self.conv3_3(x)))
#         x = self.pool(x)

#         x = F.relu(self.bn4_1(self.conv4_1(x)))
#         x = F.relu(self.bn4_2(self.conv4_2(x)))
#         x = F.relu(self.bn4_3(self.conv4_3(x)))
#         x = self.pool(x)

#         x = x.view(-1, 512 * 7 * 7)
#         x = F.relu(self.fc1(self.dropout(x)))
#         x = self.fc2(x)
#         x = x.squeeze()
#         return x


In [ ]:
class Gender_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 36, kernel_size=3)
        self.bn1 = nn.BatchNorm2d(36)
        self.conv2 = nn.Conv2d(36, 64, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3)
        self.bn5 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)

        self.fc1 = nn.Linear(512 * 4 * 4, 512)
        self.dropout = nn.Dropout(0.25)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))

        x = x.view(-1, 512 * 4 * 4)
        x = F.relu(self.fc1(self.dropout(x)))
        x = self.fc2(x)
        x = x.squeeze()
        return x


In [ ]:
age_model = Age_Net().to(device)
torchsummary.summary(age_model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 128, 224, 224]           3,584
       BatchNorm2d-2        [-1, 128, 224, 224]             256
         MaxPool2d-3        [-1, 128, 112, 112]               0
            Conv2d-4        [-1, 256, 112, 112]         295,168
       BatchNorm2d-5        [-1, 256, 112, 112]             512
         MaxPool2d-6          [-1, 256, 56, 56]               0
            Conv2d-7          [-1, 512, 56, 56]       1,180,160
       BatchNorm2d-8          [-1, 512, 56, 56]           1,024
            Conv2d-9          [-1, 256, 56, 56]         131,328
      BatchNorm2d-10          [-1, 256, 56, 56]             512
        MaxPool2d-11          [-1, 256, 28, 28]               0
AdaptiveAvgPool2d-12            [-1, 256, 1, 1]               0
          Dropout-13                  [-1, 256]               0
           Linear-14                  [

In [ ]:
gender_model = Gender_Net().to(device)
torchsummary.summary(gender_model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 36, 222, 222]           1,008
       BatchNorm2d-2         [-1, 36, 222, 222]              72
         MaxPool2d-3         [-1, 36, 110, 110]               0
            Conv2d-4         [-1, 64, 108, 108]          20,800
       BatchNorm2d-5         [-1, 64, 108, 108]             128
         MaxPool2d-6           [-1, 64, 53, 53]               0
            Conv2d-7          [-1, 128, 51, 51]          73,856
       BatchNorm2d-8          [-1, 128, 51, 51]             256
         MaxPool2d-9          [-1, 128, 25, 25]               0
           Conv2d-10          [-1, 256, 23, 23]         295,168
      BatchNorm2d-11          [-1, 256, 23, 23]             512
        MaxPool2d-12          [-1, 256, 11, 11]               0
           Conv2d-13            [-1, 512, 9, 9]       1,180,160
      BatchNorm2d-14            [-1, 51

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.optim as optim

opt_age = optim.Adam(age_model.parameters(), lr=0.0003)
age_lr_scheduler = ReduceLROnPlateau(opt_age, mode='min', verbose=True)

opt_gender = optim.Adam(gender_model.parameters(), lr=0.0003)
gender_lr_scheduler = ReduceLROnPlateau(opt_gender, mode='min', verbose=True)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.best_loss = np.inf
        self.early_stop = False
        self.counter = 0

    def __call__(self, val_loss):
        if self.best_loss - val_loss > self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print("Early stopping")

early_stopping = EarlyStopping(patience=5, verbose=True)

In [ ]:
def train_loop_age(dataloader, model, loss_fn, optimizer, epoch):
    model.train()
    size = len(dataloader.dataset)
    total_loss = 0.0
    start_time = time.time()  # 에포크 시작 시간

    for batch, (x, y) in enumerate(dataloader):
        batch_start_time = time.time()  # 배치 처리 시작 시간
        x, y = x.to(device), y['age_past'].float().to(device)
        pred = model(x)
        loss = loss_fn(pred.squeeze(), y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        batch_process_time = time.time() - batch_start_time

        if batch % 10 == 0:
            processed = (batch + 1) * len(x)
            print(f'Epoch {epoch+1} : [{processed} / {size}] loss : {loss.item()}, Batch time: {batch_process_time:.4f} sec')

    average_loss = total_loss / len(dataloader)
    epoch_time = time.time() - start_time

    print(f"Epoch {epoch+1} finished, Total Epoch time: {epoch_time:.4f} sec")
    return average_loss


In [ ]:
def validation_loop_age(dataloader, model, loss_fn, device):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y['age_past'].float().to(device)
            pred = model(x)
            loss = loss_fn(pred, y)
            val_loss += loss.item()
    val_loss /= len(dataloader)
    return val_loss

In [ ]:
def save_model(epoch, model, optimizer, path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, path)

In [ ]:
start = time.time()
for epoch in range(30):
    age_loss = train_loop_age(train_loader, age_model, nn.MSELoss(), opt_age, epoch)
    val_loss = validation_loop_age(val_loader, age_model, nn.MSELoss(), device)
    age_lr_scheduler.step(val_loss)

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        save_model(epoch, age_model, opt_age, '/content/drive/MyDrive/DL_DATA/Model/norm/age_model_norm_checkpoint.pth')
        break

    save_model(epoch, age_model, opt_age, f'/content/drive/MyDrive/DL_DATA/Model/norm/age_model_norm_checkpoint_epoch_{epoch+1}.pth')
    print(f'Epoch : {epoch+1}, Loss : {age_loss}, Val_loss : {val_loss}')

total_time = time.time() - start

# 전체 학습 시간 출력
hours, rem = divmod(total_time, 3600)
minutes, seconds = divmod(rem, 60)
print("Total training time: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

Epoch 1 : [96 / 40150] loss : 710.0734252929688, Batch time: 1.7963 sec
Epoch 1 : [1056 / 40150] loss : 691.0452880859375, Batch time: 1.1965 sec
Epoch 1 : [2016 / 40150] loss : 397.5665283203125, Batch time: 1.2037 sec
Epoch 1 : [2976 / 40150] loss : 440.4202575683594, Batch time: 1.2074 sec
Epoch 1 : [3936 / 40150] loss : 493.1903076171875, Batch time: 1.1995 sec
Epoch 1 : [4896 / 40150] loss : 213.16488647460938, Batch time: 1.1978 sec
Epoch 1 : [5856 / 40150] loss : 240.6251220703125, Batch time: 1.2018 sec


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (133505320 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 1 : [6816 / 40150] loss : 216.26339721679688, Batch time: 1.2007 sec
Epoch 1 : [7776 / 40150] loss : 259.8624572753906, Batch time: 1.1935 sec
Epoch 1 : [8736 / 40150] loss : 188.62210083007812, Batch time: 1.1980 sec
Epoch 1 : [9696 / 40150] loss : 221.4093475341797, Batch time: 1.1867 sec
Epoch 1 : [10656 / 40150] loss : 147.3946533203125, Batch time: 1.4918 sec
Epoch 1 : [11616 / 40150] loss : 193.8997802734375, Batch time: 1.4703 sec


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 1 : [12576 / 40150] loss : 192.81228637695312, Batch time: 1.4908 sec
Epoch 1 : [13536 / 40150] loss : 172.88577270507812, Batch time: 1.5024 sec
Epoch 1 : [14496 / 40150] loss : 245.886962890625, Batch time: 1.4932 sec
Epoch 1 : [15456 / 40150] loss : 165.77456665039062, Batch time: 1.4852 sec
Epoch 1 : [16416 / 40150] loss : 174.89431762695312, Batch time: 1.4945 sec
Epoch 1 : [17376 / 40150] loss : 183.56419372558594, Batch time: 1.4936 sec
Epoch 1 : [18336 / 40150] loss : 172.33642578125, Batch time: 1.4834 sec
Epoch 1 : [19296 / 40150] loss : 142.19503784179688, Batch time: 1.4921 sec
Epoch 1 : [20256 / 40150] loss : 203.39794921875, Batch time: 1.5041 sec
Epoch 1 : [21216 / 40150] loss : 243.62863159179688, Batch time: 1.4985 sec
Epoch 1 : [22176 / 40150] loss : 196.03216552734375, Batch time: 1.4927 sec
Epoch 1 : [23136 / 40150] loss : 181.86996459960938, Batch time: 1.4966 sec
Epoch 1 : [24096 / 40150] loss : 155.40586853027344, Batch time: 1.4881 sec
Epoch 1 : [25056 / 4

...

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (133505320 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 6 : [3936 / 40150] loss : 170.88584899902344, Batch time: 1.4707 sec
Epoch 6 : [4896 / 40150] loss : 111.3719482421875, Batch time: 1.5032 sec
Epoch 6 : [5856 / 40150] loss : 97.89591979980469, Batch time: 1.4959 sec
Epoch 6 : [6816 / 40150] loss : 128.2555389404297, Batch time: 1.4933 sec
Epoch 6 : [7776 / 40150] loss : 112.336669921875, Batch time: 1.4923 sec
Epoch 6 : [8736 / 40150] loss : 140.30694580078125, Batch time: 1.5205 sec
Epoch 6 : [9696 / 40150] loss : 135.0364990234375, Batch time: 1.4886 sec
Epoch 6 : [10656 / 40150] loss : 146.14891052246094, Batch time: 1.4846 sec


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 6 : [11616 / 40150] loss : 152.44442749023438, Batch time: 1.4794 sec
Epoch 6 : [12576 / 40150] loss : 119.0074691772461, Batch time: 1.5035 sec
Epoch 6 : [13536 / 40150] loss : 118.15089416503906, Batch time: 1.4782 sec
Epoch 6 : [14496 / 40150] loss : 128.25062561035156, Batch time: 1.4964 sec
Epoch 6 : [15456 / 40150] loss : 125.86541748046875, Batch time: 1.4776 sec
Epoch 6 : [16416 / 40150] loss : 111.12800598144531, Batch time: 1.5042 sec
Epoch 6 : [17376 / 40150] loss : 129.53817749023438, Batch time: 1.4832 sec
Epoch 6 : [18145 / 40150] loss : 111.55670928955078, Batch time: 1.2015 sec
Epoch 6 : [19296 / 40150] loss : 161.306396484375, Batch time: 1.4696 sec
Epoch 6 : [20256 / 40150] loss : 112.29794311523438, Batch time: 1.5019 sec
Epoch 6 : [21216 / 40150] loss : 124.41532897949219, Batch time: 1.4926 sec
Epoch 6 : [22176 / 40150] loss : 163.0113067626953, Batch time: 1.4869 sec
Epoch 6 : [23136 / 40150] loss : 153.73968505859375, Batch time: 1.4890 sec
Epoch 6 : [24096

KeyboardInterrupt: 

- Epoch 1 finished, Total Epoch time: 3335.0356 sec
- Epoch : 1, Loss : 212.2974793245229, Val_loss : 154.38576752284789
- Epoch 2 finished, Total Epoch time: 3487.7665 sec
- Epoch : 2, Loss : 158.58959261728072, Val_loss : 140.05805379039836
- Epoch 3 finished, Total Epoch time: 3397.0818 sec
- Epoch : 3, Loss : 148.9369029771172, Val_loss : 134.6221329311155
- Epoch 4 finished, Total Epoch time: 3310.7062 sec
- Epoch : 4, Loss : 141.7470114805818, Val_loss : 132.08890734978442
- Epoch 5 finished, Total Epoch time: 3321.1722 sec
- Epoch : 5, Loss : 136.13610421046437, Val_loss : 124.25044768711307